<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W7/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tuning a BERT model for text classification

BERT (Bidirectional encoder representations from transformers) is a special kind of transfomer architecture that has wide applications in NLP.

# Imports

In [11]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader,Dataset
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss,BCELoss
from torch import accelerator
from torch import tensor
from torch.nn import Sigmoid
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer


Here it's handy as well to set a variable that will hold the device we want to train the model on. That way we don't have to manually change the device if we no longer have  GPU.

In [2]:
device=accelerator.current_accelerator().type if accelerator.is_available() else "cpu"

# Hyperparameters

In [3]:
batch_size=32
lr=1e-5
epochs=5

# Data preparation and tokenisation

Our corpus is a CSV file of Amazon reviews eith three columns: label, title of the review, and the text of the review itself.

For the tokeniser, we are lucky to have a pretrained tokeniser from `AutoTokenizer`. This tokenizer returns the tokens of the corpus in a special dictionary, which will be used in the `DataLoader` class to feed the input correctly into the model. The output of the tokenizer is a dictionary with keys `input_ids`, `token_type_ids`, and `attention_mask`, so we can't just use our homeegrown tokenizer or that of `CounntVectorizer` anymore. We can, however, still use the `LabelEncoder` from `sklearn` to convert our labels into ones and zeroes.




In [4]:
corpus_file="xaa"
data=pd.read_csv(corpus_file,encoding="utf-8")
trainX,testX,trainY,testY=train_test_split(data["text"],data["label"],test_size=0.1)

label_encoder=LabelEncoder()

tokenizer=AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
train_tokens=tokenizer(list(trainX),truncation=True,padding=True)
test_tokens=tokenizer(list(testX),truncation=True,padding=True)
train_labels=label_encoder.fit_transform(trainY)
test_labels=label_encoder.transform(testY)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Loading the data

Models implemented in `torch` can be trained by feeding input data through the `DataLoader` and `DataSet` classes. This ensures that the data comes in the specified batches and that the data is also shuffled around.

To implement our own class that inherits from `DataLoader`, we just need to define a copy of basic methods: a method to return the amount of data we have, and a method to return the both the input and and the corresponding output given an index. In this case we have included the labels in the dictionary that is returned, but we could also have returned the labels separately.

In [5]:
class token_data(Dataset):
    def __init__(self,data,tokens,labels):
        # self.data will hold raw data (not tokenized) for __len__ method
        self.data=data
        self.tokens=tokens
        self.labels=labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        item={}
        for k,v in self.tokens.items():
            item[k]=tensor(v[idx])
        item["labels"]=tensor(self.labels[idx]).float()
        return item

Now that we have our  own `token_data` class, all that is required is to wrap our data using the `DataLoader` class.

In [6]:
train_loader=DataLoader(token_data(trainX,train_tokens,train_labels),shuffle=True,batch_size=batch_size)
test_loader=DataLoader(token_data(testX,test_tokens,test_labels),shuffle=True,batch_size=batch_size)

# Loading the pretrained BERT model

We use binary cross entropy loss because our classification task is binary---the reviews are either positive or negative.

In [7]:
model=BertForSequenceClassification.from_pretrained("google-bert/bert-base-cased",num_labels=1).to(device)
optimizer=Adam(model.parameters(),lr=lr)
loss_fn=BCEWithLogitsLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training


In [8]:
model.train()

for epoch in range(epochs):
    for i,batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids=batch["input_ids"].to(device)
        attention_mask=batch["attention_mask"].to(device)
        labels=batch["labels"].to(device)

        prediction=model(input_ids,attention_mask=attention_mask)
        loss=loss_fn(prediction.logits.flatten(),labels)
        loss.backward()
        optimizer.step()

        batch_loss=loss.item()
        print(f"Batch {i}, loss {batch_loss/batch_size:.6f}")
    print(f"Epoch {epoch}, loss {batch_loss/batch_size:.6f}")

Batch 0, loss 0.022684
Batch 1, loss 0.021231
Batch 2, loss 0.022359
Batch 3, loss 0.022623
Batch 4, loss 0.021243
Batch 5, loss 0.021306
Batch 6, loss 0.022828
Batch 7, loss 0.022013
Batch 8, loss 0.021723
Batch 9, loss 0.021745
Batch 10, loss 0.021823
Batch 11, loss 0.021475
Batch 12, loss 0.022177
Batch 13, loss 0.021091
Batch 14, loss 0.022031
Batch 15, loss 0.021447
Batch 16, loss 0.020649
Batch 17, loss 0.021721
Batch 18, loss 0.021265
Batch 19, loss 0.021269
Batch 20, loss 0.022242
Batch 21, loss 0.021426
Batch 22, loss 0.021647
Batch 23, loss 0.021131
Batch 24, loss 0.021343
Batch 25, loss 0.021403
Batch 26, loss 0.020722
Batch 27, loss 0.020725
Batch 28, loss 0.021114
Batch 29, loss 0.021067
Batch 30, loss 0.021079
Batch 31, loss 0.020508
Batch 32, loss 0.019848
Batch 33, loss 0.020806
Batch 34, loss 0.020772
Batch 35, loss 0.020537
Batch 36, loss 0.020341
Batch 37, loss 0.020526
Batch 38, loss 0.019632
Batch 39, loss 0.019284
Batch 40, loss 0.019157
Batch 41, loss 0.020204
Ba

KeyboardInterrupt: 

# Testing

To turn the predictions back into labels, we need to apply the `Sigmoid()` activation function on the `logits` of the prediction. This is because the `logit` function is the inverse of the `Sigmoid` function.

In [17]:
model.eval()
correct = 0
for i,batch in enumerate(test_loader):
    input_ids=batch["input_ids"].to(device)
    attention_mask=batch["attention_mask"].to(device)
    labels=batch["labels"].to(device)

    with torch.no_grad():
        prediction=model(input_ids,attention_mask=attention_mask)
    loss=loss_fn(prediction.logits.flatten(),labels)
    batch_loss=loss.item()
    print(f"Batch {i}, loss {batch_loss/batch_size:.6f}")
    correct+=(torch.round(Sigmoid()(prediction.logits.flatten()))==labels).sum().item()
    print(f"Accuracy: {correct/((i+1)*batch_size):.5f}")

Batch 0, loss 0.001448
Accuracy: 0.96875
Batch 1, loss 0.005973
Accuracy: 0.93750
Batch 2, loss 0.008603
Accuracy: 0.92708
Batch 3, loss 0.010611
Accuracy: 0.91406
Batch 4, loss 0.000710
Accuracy: 0.93125
Batch 5, loss 0.001507
Accuracy: 0.94271
Batch 6, loss 0.008799
Accuracy: 0.93750
Batch 7, loss 0.008199
Accuracy: 0.93359
Batch 8, loss 0.007214
Accuracy: 0.93403
Batch 9, loss 0.003545
Accuracy: 0.93437
Batch 10, loss 0.005889
Accuracy: 0.93466
Batch 11, loss 0.001989
Accuracy: 0.93750
Batch 12, loss 0.009513
Accuracy: 0.93510
Batch 13, loss 0.005261
Accuracy: 0.93750
Batch 14, loss 0.006473
Accuracy: 0.93750
Batch 15, loss 0.004803
Accuracy: 0.93945
Batch 16, loss 0.005058
Accuracy: 0.94118
Batch 17, loss 0.009293
Accuracy: 0.93924
Batch 18, loss 0.006372
Accuracy: 0.93586
Batch 19, loss 0.011046
Accuracy: 0.93281
Batch 20, loss 0.006026
Accuracy: 0.93304
Batch 21, loss 0.004657
Accuracy: 0.93466
Batch 22, loss 0.005616
Accuracy: 0.93342
Batch 23, loss 0.003843
Accuracy: 0.93359
Ba